In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# 샘플 벡터DB 구축
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
embedding = HuggingFaceBgeEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.get_relevant_documents(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


4

In [3]:
import os
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)


multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=db.as_retriever(),
    llm=model,
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface.chat_models import huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [5]:
# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
# 문서 검색
relevant_docs = multiquery_retriever.get_relevant_documents(query=question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three alternative versions of the user question to retrieve relevant documents from a vector database:', '', 'What are the available functions of the OpenAI Assistant API and how do I use them?', '', "Can you provide an overview of the OpenAI Assistant API's functionality and its usage guidelines?", '', 'How do I utilize the various functions offered by the OpenAI Assistant API, and what are the best practices for implementing them?', '', 'These alternative questions aim to capture different aspects of the original question, such as:', '', '* The first question focuses on the specific functions offered by the API and their usage.', "* The second question emphasizes the need for an overview of the API's functionality and usage guidelines.", "* The third question highlights the practical aspect of utilizing the API's functions and best practices for implementation.", '', 'By generating these alternative questions, we can

검색된 문서 개수: 19
이번에 진행할 예제는 퀴즈문제 출제 챗봇입니다.
우리의 수학 튜터를 위한 display_quiz() 함수를 생성하였습니다(사실 이 함수는 퀴즈 문제에 대한 사용자 입력을 받는 함수라 역할이 중요하지는 않습니다).
이 함수는 title과 question의 배열을 받아 퀴즈를 표시하고 사용자로부터 각각에 대한 입력을 받습니다.


title


questions


question_text


choices: [“선택지 1”, “선택지 2”, “선택지 3”, “선택지 4”]




def display_quiz(title, questions, show_numeric=False):
    print(f"제목: {title}\n")
    responses = []

    for q in questions:
        # 질문을 출력합니다.
        print(q["question_text"])
        response = ""


In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}
"""
)

# LLMChain을 생성합니다.
chain = {"question": RunnablePassthrough()} | prompt | model | StrOutputParser()

# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = chain.invoke(question)
# 결과를 확인합니다.(5개 질문 생성)
print(multi_queries)

Here are five different versions of the original question to retrieve relevant documents from a vector database:

OpenAI Assistant API의 Functions 사용법에 대해 설명해주세요.
How do I use the functions of the OpenAI Assistant API?
What are the functions of the OpenAI Assistant API and how do I use them?
Can you explain the usage of the functions in the OpenAI Assistant API?
What is the usage of the functions in the OpenAI Assistant API and how do I implement it?

These variations aim to capture different aspects of the original question, such as:

* Using different verb forms (e.g., "explain" instead of "know")
* Adding more specificity (e.g., "functions" instead of just "API")
* Changing the tone and style (e.g., from formal to informal)
* Adding more context (e.g., "implementation" instead of just "usage")

By generating these variations, we can increase the chances of retrieving relevant documents that may not have been matched by the original question alone.


In [9]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=chain, retriever=db.as_retriever()
)

In [10]:
# 결과
relevant_docs = multiquery_retriever.get_relevant_documents(query=question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are five different versions of the original question:', '', 'What are the key functions of the OpenAI Assistant API, and how can I utilize them to generate multiple perspectives on a user query?', '', "Can you provide examples of how to use the OpenAI Assistant API's functions to overcome the limitations of distance-based similarity search in a vector database?", '', "How can I leverage the OpenAI Assistant API's capabilities to generate alternative questions that can help users retrieve relevant documents from a vector database?", '', "What are some best practices for using the OpenAI Assistant API's functions to generate multiple perspectives on a user query, and how can this help improve the accuracy of search results?", '', "Can you demonstrate how to use the OpenAI Assistant API's functions to generate alternative questions that can help users overcome the limitations of distance-based similarity search in a vector da

검색된 문서 개수: 9
{'question_text': '삼성전자가 공개한 자체 개발 생성 AI의 이름은 무엇인가요?',
   'choices': ['1. 삼성 뉴럴', '2. 삼성 브레인', '3. 삼성 가우스', '4. 삼성 인텔리']}]}
